# Multi-Class Classification with iris flowers dataset

## Problem Description

In this tutorial we will use the standard machine learning problem called the [iris flowers dataset](http://archive.ics.uci.edu/ml/datasets/Iris).

This dataset is well studied and is a good problem for practicing on neural networks because all of the 4 input variables are numeric and have the same scale in centimeters. Each instance describes the properties of an observed flower measurements and the output variable is specific iris species.

This is a multi-class classification problem, meaning that there are more than two classes to be predicted, in fact there are three flower species. This is an important type of problem on which to practice with neural networks because the three class values require specialized handling.

The iris flower dataset is a well studied problem and a such we can expect to achieve an model accuracy in the range of 95% to 97%. This provides a good target to aim for when developing our models.

You can [download the iris flowers dataset](http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data) from the UCI Machine Learning repository and place it in your current working directory with the filename iris.csv

## Import Classes and Functions

We can begin by importing all of the classes and functions we will need in this tutorial.

This includes both the functionality we require from Keras, but also data loading from [pandas](http://pandas.pydata.org/) as well as data preparation and model evaluation from [scikit-learn](http://scikit-learn.org/).

In [3]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

Using TensorFlow backend.


## Initialize Random Number Generator

Next we need to initialize the random number generator to a constant value (7).

This is important to ensure that the results we achieve from this model can be achieved again precisely. It ensures that the stochastic process of training a neural network model can be reproduced.

In [4]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# set some variables
batch_size = 5
epochs = 200

## Load The Dataset

The dataset can be loaded directly. Because the output variable contains strings, it is easiest to load the data using pandas. We can then split the attributes (columns) into input variables (X) and output variables (Y).

In [5]:
# load dataset
dataframe = pd.read_csv("iris.csv", header=None)
dataframe.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
dataset = dataframe.values
X_data = dataset[:,0:4].astype(float)
y_data = dataset[:,4]

In [7]:
print("X matrix shape", X_data.shape)
print("Y matrix shape", y_data.shape)

X matrix shape (150, 4)
Y matrix shape (150,)


# Encode The Output Variable

The output variable contains three different string values.

When modeling multi-class classification problems using neural networks, it is good practice to reshape the output attribute from a vector that contains values for each class value to be a matrix with a boolean for each class value and whether or not a given instance has that class value or not.

This is called [one hot encoding](https://en.wikipedia.org/wiki/One-hot) or creating dummy variables from a categorical variable.

For example, in this problem three class values are Iris-setosa, Iris-versicolor and Iris-virginica. If we had the observations:

* Iris-setosa
* Iris-versicolor
* Iris-virginica

We can turn this into a one-hot encoded binary matrix for each data instance that would look as follows:

<pre>

| Iris-setosa | Iris-versicolor | Iris-virginica |
|-------------|-----------------|----------------|
|      1      |        0        |       0        |
|-------------|-----------------|----------------|
|      0      |        1        |       0        |
|-------------|-----------------|----------------|
|      0      |        0        |       1        |

</pre>

We can do this by first encoding the strings consistently to integers using the scikit-learn class LabelEncoder. Then convert the vector of integers to a one hot encoding using the Keras function to_categorical().

In [6]:
uniques, ids = np.unique(y_data, return_inverse=True)
Y_data = np_utils.to_categorical(ids, len(uniques))

## Define The Neural Network Model

The Keras library provides wrapper classes to allow you to use neural network models developed with Keras in scikit-learn.

There is a KerasClassifier class in Keras that can be used as an Estimator in scikit-learn, the base type of model in the library. The KerasClassifier takes the name of a function as an argument. This function must return the constructed neural network model, ready for training.

Below is a function that will create a baseline neural network for the iris classification problem. It creates a simple fully connected network with one hidden layer that contains 4 neurons, the same number of inputs (it could be any number of neurons).

The hidden layer uses a rectifier activation function which is a good practice. Because we used a one-hot encoding for our iris dataset, the output layer must create 3 output values, one for each class. The output value with the largest value will be taken as the class predicted by the model.

The network topology of this simple one-layer neural network can be summarized as:

* inputs -> [4 hidden nodes] -> 3 outputs

Note that we use a sigmoid activation function in the output layer. This is to ensure the output values are in the range of 0 and 1 and may be used as predicted probabilities.

Finally, the network uses the efficient ADAM gradient descent optimization algorithm with a logarithmic loss function, which is called categorical_crossentropy in Keras.

In [7]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, kernel_initializer='normal'))
    model.add(Activation('sigmoid'))
    model.add(Dense(3, kernel_initializer='normal'))
    model.add(Activation('softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

We can now create our KerasClassifier for use in scikit-learn.

We can also pass arguments in the construction of the KerasClassifier class that will be passed on to the fit() function internally used to train the neural network. Here, we pass the number of epochs as 200 and batch size as 5 to use when training the model. Debugging is also turned off when training by setting verbose to 0.

In [8]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=epochs, batch_size=batch_size, verbose=0)

## Evaluate The Model with k-Fold Cross Validation

We can now evaluate the neural network model on our training data.

The scikit-learn has excellent capability to evaluate models using a suite of techniques. The gold standard for evaluating machine learning models is k-fold cross validation.

First we can define the model evaluation procedure.  
Here, we set the number of folds to be 10 (an excellent default) and to shuffle the data before partitioning it.

In [9]:
kfold = KFold(n_splits=10, shuffle=True, random_state=None)

Now we can evaluate our model (estimator) on our dataset (X_data and Y_data) using a 10-fold cross validation procedure (kfold).

Evaluating the model only takes approximately 10 seconds and returns an object that describes the evaluation of the 10 constructed models for each of the splits of the dataset.

In [10]:
results = cross_val_score(estimator, X_data, Y_data, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/200
0s - loss: 1.1001 - acc: 0.3333
Epoch 2/200
0s - loss: 1.0972 - acc: 0.3333
Epoch 3/200
0s - loss: 1.0956 - acc: 0.3333
Epoch 4/200
0s - loss: 1.0938 - acc: 0.3333
Epoch 5/200
0s - loss: 1.0915 - acc: 0.3333
Epoch 6/200
0s - loss: 1.0890 - acc: 0.3333
Epoch 7/200
0s - loss: 1.0861 - acc: 0.4148
Epoch 8/200
0s - loss: 1.0823 - acc: 0.6741
Epoch 9/200
0s - loss: 1.0780 - acc: 0.6815
Epoch 10/200
0s - loss: 1.0730 - acc: 0.6815
Epoch 11/200
0s - loss: 1.0662 - acc: 0.6815
Epoch 12/200
0s - loss: 1.0581 - acc: 0.6815
Epoch 13/200
0s - loss: 1.0494 - acc: 0.6815
Epoch 14/200
0s - loss: 1.0388 - acc: 0.6815
Epoch 15/200
0s - loss: 1.0273 - acc: 0.6815
Epoch 16/200
0s - loss: 1.0140 - acc: 0.6815
Epoch 17/200
0s - loss: 0.9994 - acc: 0.6815
Epoch 18/200
0s - loss: 0.9835 - acc: 0.6815
Epoch 19/200
0s - loss: 0.9674 - acc: 0.6815
Epoch 20/200
0s - loss: 0.9498 - acc: 0.6815
Epoch 21/200
0s - loss: 0.9316 - acc: 0.6815
Epoch 22/200
0s - loss: 0.9136 - acc: 0.6815
Epoch 23/200
0s - l

0s - loss: 0.1861 - acc: 0.9704
Epoch 186/200
0s - loss: 0.1843 - acc: 0.9704
Epoch 187/200
0s - loss: 0.1834 - acc: 0.9704
Epoch 188/200
0s - loss: 0.1822 - acc: 0.9704
Epoch 189/200
0s - loss: 0.1816 - acc: 0.9704
Epoch 190/200
0s - loss: 0.1804 - acc: 0.9630
Epoch 191/200
0s - loss: 0.1770 - acc: 0.9704
Epoch 192/200
0s - loss: 0.1762 - acc: 0.9704
Epoch 193/200
0s - loss: 0.1746 - acc: 0.9704
Epoch 194/200
0s - loss: 0.1738 - acc: 0.9704
Epoch 195/200
0s - loss: 0.1732 - acc: 0.9630
Epoch 196/200
0s - loss: 0.1714 - acc: 0.9704
Epoch 197/200
0s - loss: 0.1695 - acc: 0.9704
Epoch 198/200
0s - loss: 0.1687 - acc: 0.9704
Epoch 199/200
0s - loss: 0.1677 - acc: 0.9704
Epoch 200/200
0s - loss: 0.1669 - acc: 0.9630
Epoch 1/200
0s - loss: 1.0975 - acc: 0.3333
Epoch 2/200
0s - loss: 1.0941 - acc: 0.3333
Epoch 3/200
0s - loss: 1.0915 - acc: 0.3333
Epoch 4/200
0s - loss: 1.0887 - acc: 0.5704
Epoch 5/200
0s - loss: 1.0846 - acc: 0.6889
Epoch 6/200
0s - loss: 1.0803 - acc: 0.6741
Epoch 7/200
0s

0s - loss: 0.2078 - acc: 0.9704
Epoch 170/200
0s - loss: 0.2074 - acc: 0.9704
Epoch 171/200
0s - loss: 0.2043 - acc: 0.9704
Epoch 172/200
0s - loss: 0.2028 - acc: 0.9704
Epoch 173/200
0s - loss: 0.2016 - acc: 0.9704
Epoch 174/200
0s - loss: 0.2006 - acc: 0.9778
Epoch 175/200
0s - loss: 0.1984 - acc: 0.9704
Epoch 176/200
0s - loss: 0.1976 - acc: 0.9704
Epoch 177/200
0s - loss: 0.1955 - acc: 0.9704
Epoch 178/200
0s - loss: 0.1947 - acc: 0.9704
Epoch 179/200
0s - loss: 0.1936 - acc: 0.9704
Epoch 180/200
0s - loss: 0.1919 - acc: 0.9704
Epoch 181/200
0s - loss: 0.1907 - acc: 0.9704
Epoch 182/200
0s - loss: 0.1887 - acc: 0.9704
Epoch 183/200
0s - loss: 0.1876 - acc: 0.9704
Epoch 184/200
0s - loss: 0.1864 - acc: 0.9704
Epoch 185/200
0s - loss: 0.1846 - acc: 0.9704
Epoch 186/200
0s - loss: 0.1847 - acc: 0.9852
Epoch 187/200
0s - loss: 0.1823 - acc: 0.9704
Epoch 188/200
0s - loss: 0.1812 - acc: 0.9704
Epoch 189/200
0s - loss: 0.1798 - acc: 0.9704
Epoch 190/200
0s - loss: 0.1787 - acc: 0.9704
Ep

0s - loss: 0.2507 - acc: 0.9704
Epoch 151/200
0s - loss: 0.2497 - acc: 0.9704
Epoch 152/200
0s - loss: 0.2462 - acc: 0.9704
Epoch 153/200
0s - loss: 0.2450 - acc: 0.9704
Epoch 154/200
0s - loss: 0.2433 - acc: 0.9704
Epoch 155/200
0s - loss: 0.2413 - acc: 0.9704
Epoch 156/200
0s - loss: 0.2396 - acc: 0.9704
Epoch 157/200
0s - loss: 0.2377 - acc: 0.9704
Epoch 158/200
0s - loss: 0.2358 - acc: 0.9704
Epoch 159/200
0s - loss: 0.2339 - acc: 0.9704
Epoch 160/200
0s - loss: 0.2327 - acc: 0.9704
Epoch 161/200
0s - loss: 0.2325 - acc: 0.9704
Epoch 162/200
0s - loss: 0.2301 - acc: 0.9704
Epoch 163/200
0s - loss: 0.2290 - acc: 0.9778
Epoch 164/200
0s - loss: 0.2258 - acc: 0.9704
Epoch 165/200
0s - loss: 0.2244 - acc: 0.9704
Epoch 166/200
0s - loss: 0.2227 - acc: 0.9704
Epoch 167/200
0s - loss: 0.2220 - acc: 0.9630
Epoch 168/200
0s - loss: 0.2197 - acc: 0.9704
Epoch 169/200
0s - loss: 0.2185 - acc: 0.9704
Epoch 170/200
0s - loss: 0.2165 - acc: 0.9704
Epoch 171/200
0s - loss: 0.2159 - acc: 0.9704
Ep

0s - loss: 0.3036 - acc: 0.9704
Epoch 134/200
0s - loss: 0.3020 - acc: 0.9704
Epoch 135/200
0s - loss: 0.2988 - acc: 0.9704
Epoch 136/200
0s - loss: 0.2970 - acc: 0.9704
Epoch 137/200
0s - loss: 0.2950 - acc: 0.9704
Epoch 138/200
0s - loss: 0.2920 - acc: 0.9704
Epoch 139/200
0s - loss: 0.2900 - acc: 0.9704
Epoch 140/200
0s - loss: 0.2878 - acc: 0.9704
Epoch 141/200
0s - loss: 0.2859 - acc: 0.9704
Epoch 142/200
0s - loss: 0.2837 - acc: 0.9704
Epoch 143/200
0s - loss: 0.2812 - acc: 0.9704
Epoch 144/200
0s - loss: 0.2796 - acc: 0.9704
Epoch 145/200
0s - loss: 0.2771 - acc: 0.9704
Epoch 146/200
0s - loss: 0.2770 - acc: 0.9704
Epoch 147/200
0s - loss: 0.2729 - acc: 0.9704
Epoch 148/200
0s - loss: 0.2714 - acc: 0.9704
Epoch 149/200
0s - loss: 0.2695 - acc: 0.9704
Epoch 150/200
0s - loss: 0.2671 - acc: 0.9704
Epoch 151/200
0s - loss: 0.2652 - acc: 0.9704
Epoch 152/200
0s - loss: 0.2635 - acc: 0.9630
Epoch 153/200
0s - loss: 0.2617 - acc: 0.9704
Epoch 154/200
0s - loss: 0.2591 - acc: 0.9704
Ep

...

0s - loss: 0.4599 - acc: 0.9630
Epoch 99/200
0s - loss: 0.4561 - acc: 0.9630
Epoch 100/200
0s - loss: 0.4522 - acc: 0.9704
Epoch 101/200
0s - loss: 0.4487 - acc: 0.9704
Epoch 102/200
0s - loss: 0.4452 - acc: 0.9704
Epoch 103/200
0s - loss: 0.4411 - acc: 0.9704
Epoch 104/200
0s - loss: 0.4389 - acc: 0.9704
Epoch 105/200
0s - loss: 0.4343 - acc: 0.9778
Epoch 106/200
0s - loss: 0.4309 - acc: 0.9704
Epoch 107/200
0s - loss: 0.4281 - acc: 0.9704
Epoch 108/200
0s - loss: 0.4250 - acc: 0.9704
Epoch 109/200
0s - loss: 0.4195 - acc: 0.9704
Epoch 110/200
0s - loss: 0.4176 - acc: 0.9704
Epoch 111/200
0s - loss: 0.4134 - acc: 0.9704
Epoch 112/200
0s - loss: 0.4102 - acc: 0.9704
Epoch 113/200
0s - loss: 0.4068 - acc: 0.9704
Epoch 114/200
0s - loss: 0.4037 - acc: 0.9630
Epoch 115/200
0s - loss: 0.3996 - acc: 0.9778
Epoch 116/200
0s - loss: 0.3961 - acc: 0.9778
Epoch 117/200
0s - loss: 0.3930 - acc: 0.9778
Epoch 118/200
0s - loss: 0.3899 - acc: 0.9778
Epoch 119/200
0s - loss: 0.3866 - acc: 0.9778
Epo

0s - loss: 0.5078 - acc: 0.9778
Epoch 81/200
0s - loss: 0.5046 - acc: 0.9630
Epoch 82/200
0s - loss: 0.5015 - acc: 0.9778
Epoch 83/200
0s - loss: 0.4987 - acc: 0.9704
Epoch 84/200
0s - loss: 0.4956 - acc: 0.9630
Epoch 85/200
0s - loss: 0.4923 - acc: 0.9778
Epoch 86/200
0s - loss: 0.4892 - acc: 0.9778
Epoch 87/200
0s - loss: 0.4865 - acc: 0.9630
Epoch 88/200
0s - loss: 0.4831 - acc: 0.9630
Epoch 89/200
0s - loss: 0.4802 - acc: 0.9630
Epoch 90/200
0s - loss: 0.4769 - acc: 0.9778
Epoch 91/200
0s - loss: 0.4738 - acc: 0.9778
Epoch 92/200
0s - loss: 0.4706 - acc: 0.9556
Epoch 93/200
0s - loss: 0.4672 - acc: 0.9630
Epoch 94/200
0s - loss: 0.4638 - acc: 0.9630
Epoch 95/200
0s - loss: 0.4607 - acc: 0.9704
Epoch 96/200
0s - loss: 0.4578 - acc: 0.9630
Epoch 97/200
0s - loss: 0.4539 - acc: 0.9630
Epoch 98/200
0s - loss: 0.4506 - acc: 0.9630
Epoch 99/200
0s - loss: 0.4473 - acc: 0.9704
Epoch 100/200
0s - loss: 0.4437 - acc: 0.9630
Epoch 101/200
0s - loss: 0.4403 - acc: 0.9630
Epoch 102/200
0s - lo

0s - loss: 0.5134 - acc: 0.9481
Epoch 62/200
0s - loss: 0.5098 - acc: 0.9407
Epoch 63/200
0s - loss: 0.5057 - acc: 0.9630
Epoch 64/200
0s - loss: 0.5017 - acc: 0.9630
Epoch 65/200
0s - loss: 0.4977 - acc: 0.9704
Epoch 66/200
0s - loss: 0.4939 - acc: 0.9704
Epoch 67/200
0s - loss: 0.4903 - acc: 0.9704
Epoch 68/200
0s - loss: 0.4856 - acc: 0.9704
Epoch 69/200
0s - loss: 0.4819 - acc: 0.9704
Epoch 70/200
0s - loss: 0.4781 - acc: 0.9704
Epoch 71/200
0s - loss: 0.4744 - acc: 0.9704
Epoch 72/200
0s - loss: 0.4706 - acc: 0.9704
Epoch 73/200
0s - loss: 0.4662 - acc: 0.9704
Epoch 74/200
0s - loss: 0.4626 - acc: 0.9704
Epoch 75/200
0s - loss: 0.4586 - acc: 0.9704
Epoch 76/200
0s - loss: 0.4556 - acc: 0.9704
Epoch 77/200
0s - loss: 0.4509 - acc: 0.9778
Epoch 78/200
0s - loss: 0.4479 - acc: 0.9778
Epoch 79/200
0s - loss: 0.4435 - acc: 0.9704
Epoch 80/200
0s - loss: 0.4398 - acc: 0.9704
Epoch 81/200
0s - loss: 0.4356 - acc: 0.9778
Epoch 82/200
0s - loss: 0.4323 - acc: 0.9778
Epoch 83/200
0s - loss:

0s - loss: 0.6072 - acc: 0.6963
Epoch 45/200
0s - loss: 0.6009 - acc: 0.7037
Epoch 46/200
0s - loss: 0.5950 - acc: 0.7111
Epoch 47/200
0s - loss: 0.5896 - acc: 0.7111
Epoch 48/200
0s - loss: 0.5840 - acc: 0.7111
Epoch 49/200
0s - loss: 0.5790 - acc: 0.7111
Epoch 50/200
0s - loss: 0.5741 - acc: 0.7111
Epoch 51/200
0s - loss: 0.5694 - acc: 0.7111
Epoch 52/200
0s - loss: 0.5649 - acc: 0.7185
Epoch 53/200
0s - loss: 0.5604 - acc: 0.7185
Epoch 54/200
0s - loss: 0.5561 - acc: 0.7259
Epoch 55/200
0s - loss: 0.5522 - acc: 0.7185
Epoch 56/200
0s - loss: 0.5483 - acc: 0.7259
Epoch 57/200
0s - loss: 0.5443 - acc: 0.7333
Epoch 58/200
0s - loss: 0.5405 - acc: 0.7481
Epoch 59/200
0s - loss: 0.5367 - acc: 0.7704
Epoch 60/200
0s - loss: 0.5336 - acc: 0.7852
Epoch 61/200
0s - loss: 0.5294 - acc: 0.8000
Epoch 62/200
0s - loss: 0.5262 - acc: 0.8148
Epoch 63/200
0s - loss: 0.5223 - acc: 0.8222
Epoch 64/200
0s - loss: 0.5189 - acc: 0.8222
Epoch 65/200
0s - loss: 0.5152 - acc: 0.8667
Epoch 66/200
0s - loss:

0s - loss: 0.8816 - acc: 0.6889
Epoch 26/200
0s - loss: 0.8637 - acc: 0.6963
Epoch 27/200
0s - loss: 0.8463 - acc: 0.6889
Epoch 28/200
0s - loss: 0.8287 - acc: 0.6963
Epoch 29/200
0s - loss: 0.8114 - acc: 0.6963
Epoch 30/200
0s - loss: 0.7950 - acc: 0.7037
Epoch 31/200
0s - loss: 0.7786 - acc: 0.7037
Epoch 32/200
0s - loss: 0.7638 - acc: 0.6963
Epoch 33/200
0s - loss: 0.7489 - acc: 0.7111
Epoch 34/200
0s - loss: 0.7344 - acc: 0.7111
Epoch 35/200
0s - loss: 0.7206 - acc: 0.7111
Epoch 36/200
0s - loss: 0.7077 - acc: 0.7111
Epoch 37/200
0s - loss: 0.6957 - acc: 0.6963
Epoch 38/200
0s - loss: 0.6839 - acc: 0.7037
Epoch 39/200
0s - loss: 0.6726 - acc: 0.7111
Epoch 40/200
0s - loss: 0.6620 - acc: 0.7111
Epoch 41/200
0s - loss: 0.6521 - acc: 0.7111
Epoch 42/200
0s - loss: 0.6435 - acc: 0.7037
Epoch 43/200
0s - loss: 0.6339 - acc: 0.7037
Epoch 44/200
0s - loss: 0.6253 - acc: 0.7185
Epoch 45/200
0s - loss: 0.6173 - acc: 0.7481
Epoch 46/200
0s - loss: 0.6098 - acc: 0.7556
Epoch 47/200
0s - loss:

The results are summarized as both the mean and standard deviation of the model accuracy on the dataset. This is a reasonable estimation of the performance of the model on unseen data. It is also within the realm of known top results for this problem.